In [21]:
import requests
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv

root_path = os.path.dirname(os.getcwd())
file_env = f'{root_path}/environments/.env'
load_dotenv(dotenv_path=file_env)

URL_OPENWEATHERMAP_API = os.getenv("URL_OPENWEATHERMAP_API")
API_OPENWEATHERMAP_KEY = os.getenv("API_OPENWEATHERMAP_KEY")

class openweathermap_api:

    def __init__(self) -> None:
        n = 300
        self.base_url = f"{URL_OPENWEATHERMAP_API}"
        self.api_key = API_OPENWEATHERMAP_KEY
        # Mexico
        self.latitudes = np.random.uniform(low=14.5, high=32.5, size=n)
        self.longitudes = np.random.uniform(low=-118.5, high=-86.5, size=n)

    def extract_data(self):
        weather_data = []
        for lat, lon in zip(self.latitudes, self.longitudes):
            params = {'lat': lat, 'lon': lon, 'appid': self.api_key, 'lang': "es"}
            response = requests.get(self.base_url, params=params)
            if response.status_code == 200:
                weather_data.append(response.json())
            else:
                print(f"Error lat={lat}, lon={lon}")
        return weather_data
    

    def transform_data(self, weather_data):
        data = []
        for city in weather_data:
            data.append({
                'city': city['name'],
                'country': city['sys'].get('country', 'N/A'),
                'latitude': city['coord']['lat'],
                'longitude': city['coord']['lon'],
                'weather': city['weather'][0]['description'],
                'temperature': city['main']['temp'],
                'pressure': city['main']['pressure'],
                'humidity': city['main']['humidity'],
                'wind_speed': city['wind']['speed'],
                'cloudiness': city['clouds']['all'],
                'sunrise': city['sys']['sunrise'],
                'sunset': city['sys']['sunset']
            })
        return data
    
    def load_data(self, data):
        df = pd.DataFrame(data)
        # df.to_csv('data/openweathermap_raw.csv', index=False) # root path
        print("Data saved in data/openweathermap_raw.csv")
        return df
    
    def run(self):
        weather_data = self.extract_data()
        data = self.transform_data(weather_data)
        df = self.load_data(data)
        return df
    
if __name__ == '__main__':
    openweathermap = openweathermap_api()
    openweathermap.run()

OSError: Cannot save file into a non-existent directory: 'data'